# Softmax

In [1]:
import numpy as np

In [6]:
vector = np.array([11, 29, 22])

In [7]:
def softmax(vec):
    vec -= np.max(vec)#减去最大值，减少计算溢出
    exp = np.exp(vec)
    return exp / np.sum(exp)

In [8]:
softmax(vector)

array([1.52161042e-08, 9.99088934e-01, 9.11051181e-04])

# 中文分词

# Assignment-04 基于维基百科的词向量构建
在本章，你将使用Gensim和维基百科获得你的第一批词向量，并且感受词向量的基本过程。



## Step-01: Download Wikipedia Chinese Corpus: https://dumps.wikimedia.org/zhwiki/20190720/
第一步：使用维基百科下载中文语料库

## Step-02: Using https://github.com/attardi/wikiextractor to extract the wikipedia corpus
### 第二步：使用python wikipedia extractor抽取维基百科的内容

## Step-03: Using gensim get word vectors:
Reference:

https://radimrehurek.com/gensim/models/word2vec.html
https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne
### 第三步：参考Gensim的文档和Kaggle的参考文档，获得词向量。 注意，你要使用Jieba分词把维基百科的内容切分成一个一个单词，然后存进新的文件中。然后，你需要用Gensim的LineSentence这个类进行文件的读取。

此处，我使用了opencc工具将繁体字转换为简体字，生成txt。再进行数据清洗。
再训练成词向量Model.

## Step-04: Using some words to test your preformance.
第四步，测试同义词，找几个单词。

## Step-05: Using visualization tools: https://www.kaggle.com/jeffd23/visualizing-word-vectors-with-t-sne¶
第五步：使用Kaggle给出的T-SEN进行词向量的可视化。

### 语料清洗
经过简体化的文档，仍然有很多脏信息。如数字、标点符号、非中文语言字符等，并且文档中的句子是不能用来训练的，需要进行分词处理。 故编写代码，进行非中文字符串的清除，以及分词。
分词及文本清洗的代码如下，此处使用jieba分词工具。

In [22]:
#encoding: utf-8
import jieba
import os
import codecs
from tqdm import tqdm


class MySentences(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname),encoding='utf-8'):
                if len(line) > 0:
                    yield [segment.strip() for segment in jieba.cut(line.strip(), cut_all=False)
                           if segment not in stoplist and len(segment) > 0]
 
 
def is_ustr(instr): #将所有字符过滤，只保留汉字
    out_str = ''
    for index in range(len(instr)):
        if is_uchar(instr[index]):
            out_str = out_str + instr[index].strip()
    return out_str
 
 
def is_uchar(uchar):
    # """判断一个unicode是否是汉字"""
    if u'\u4e00' <= uchar <= u'\u9fff':
        return True
 
if __name__ == '__main__':
    dirname = 'D:\code_project\data\wiki_corpus\simplified_Chinese\simplify_ch'
    # 读取停用词；
    stop_f = codecs.open(u'D:\code_project\data\wiki_corpus\stop_words\stop_words.txt', 'r', encoding='gb18030')
    stoplist = {}.fromkeys([line.strip() for line in stop_f])
    # 进行jieba分词
    sentences = MySentences(dirname)
    # 分词结果写入文件
    f = codecs.open(u'D:\code_project\data\wiki_corpus\wiki_clean\wiki_jieba.txt', 'w', encoding='utf-8')
    i = 0
    j = 0
    w = tqdm(sentences, desc=u'分词句子') #内嵌进度条
    for sentence in w:
        if len(sentence) > 0:
            output = " "
            for d in sentence:
                # 去除停用词；
                if d not in stoplist:
                    output += is_ustr(d).strip() + " "
            f.write(output.strip())
            f.write('\r\n')
            i += 1
            if i % 10000 == 0: 
                j += 1
                w.set_description(u'已分词： %s万个句子'%j)
    f.close()

分词句子: 0it [00:00, ?it/s]Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\HSUHUI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.046 seconds.
Prefix dict has been built succesfully.
已分词： 764万个句子: : 15190604it [1:17:53, 3250.36it/s]


## 词向量训练
经过那么多步骤的预处理，终于得到“干净”的语料，可以开始训练了。 

In [23]:
# -*- coding: utf-8 -*-
import logging
import multiprocessing
import codecs
from tqdm import tqdm
 
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
if __name__ == '__main__':
 
    program = "train_word2vec_model.py"
    logger = logging.getLogger(program)
 
    logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
    logging.root.setLevel(level=logging.INFO)
    logger.info("running %s" % ' '.join(program))
 
    infile = "D:\code_project\data\wiki_corpus\wiki_clean\wiki_jieba.txt"
    vec_outfile1 = "D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model"
    vec_outfile2 = "D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.vector"
    sentences = LineSentence(infile)
 
    model = Word2Vec(LineSentence(infile), size=300, window=5, min_count=5,
                     workers=multiprocessing.cpu_count(), iter=2) #iter原来等于100
    # trim unneeded model memory = use(much) less RAM
    # model.init_sims(replace=True)
    model.save(vec_outfile1)
    model.wv.save_word2vec_format(vec_outfile2, binary=False)

D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-08-05 00:30:26,809: INFO: running t r a i n _ w o r d 2 v e c _ m o d e l . p y
2019-08-05 00:30:26,811: INFO: collecting all words and their counts
D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-08-05 00:30:26,823: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-08-05 00:30:27,046: INFO: PROGRESS: at sentence #10000, processed 515985 words, keeping 68257 word types
2019-08-05 00:30:27,227: INFO: PROGRESS: a

2019-08-05 00:30:38,269: INFO: PROGRESS: at sentence #660000, processed 22702203 words, keeping 670933 word types
2019-08-05 00:30:38,408: INFO: PROGRESS: at sentence #670000, processed 22974705 words, keeping 676151 word types
2019-08-05 00:30:38,564: INFO: PROGRESS: at sentence #680000, processed 23274910 words, keeping 682287 word types
2019-08-05 00:30:38,720: INFO: PROGRESS: at sentence #690000, processed 23566818 words, keeping 687389 word types
2019-08-05 00:30:38,878: INFO: PROGRESS: at sentence #700000, processed 23865152 words, keeping 693026 word types
2019-08-05 00:30:39,040: INFO: PROGRESS: at sentence #710000, processed 24146936 words, keeping 697986 word types
2019-08-05 00:30:39,278: INFO: PROGRESS: at sentence #720000, processed 24447343 words, keeping 703066 word types
2019-08-05 00:30:39,475: INFO: PROGRESS: at sentence #730000, processed 24747906 words, keeping 708792 word types
2019-08-05 00:30:39,677: INFO: PROGRESS: at sentence #740000, processed 25040290 words, 

2019-08-05 00:30:49,025: INFO: PROGRESS: at sentence #1380000, processed 42260587 words, keeping 987134 word types
2019-08-05 00:30:49,151: INFO: PROGRESS: at sentence #1390000, processed 42497159 words, keeping 990112 word types
2019-08-05 00:30:49,288: INFO: PROGRESS: at sentence #1400000, processed 42744014 words, keeping 993647 word types
2019-08-05 00:30:49,426: INFO: PROGRESS: at sentence #1410000, processed 43002272 words, keeping 997423 word types
2019-08-05 00:30:49,562: INFO: PROGRESS: at sentence #1420000, processed 43266535 words, keeping 1000806 word types
2019-08-05 00:30:49,705: INFO: PROGRESS: at sentence #1430000, processed 43536989 words, keeping 1004664 word types
2019-08-05 00:30:49,847: INFO: PROGRESS: at sentence #1440000, processed 43794617 words, keeping 1008143 word types
2019-08-05 00:30:49,989: INFO: PROGRESS: at sentence #1450000, processed 44055567 words, keeping 1011573 word types
2019-08-05 00:30:50,124: INFO: PROGRESS: at sentence #1460000, processed 443

2019-08-05 00:30:58,127: INFO: PROGRESS: at sentence #2090000, processed 58898766 words, keeping 1206543 word types
2019-08-05 00:30:58,217: INFO: PROGRESS: at sentence #2100000, processed 59065572 words, keeping 1208951 word types
2019-08-05 00:30:58,340: INFO: PROGRESS: at sentence #2110000, processed 59284549 words, keeping 1211872 word types
2019-08-05 00:30:58,471: INFO: PROGRESS: at sentence #2120000, processed 59525736 words, keeping 1217945 word types
2019-08-05 00:30:58,614: INFO: PROGRESS: at sentence #2130000, processed 59789960 words, keeping 1221376 word types
2019-08-05 00:30:58,736: INFO: PROGRESS: at sentence #2140000, processed 60012055 words, keeping 1224481 word types
2019-08-05 00:30:58,874: INFO: PROGRESS: at sentence #2150000, processed 60259427 words, keeping 1227321 word types
2019-08-05 00:30:58,999: INFO: PROGRESS: at sentence #2160000, processed 60488368 words, keeping 1229916 word types
2019-08-05 00:30:59,138: INFO: PROGRESS: at sentence #2170000, processed

2019-08-05 00:31:07,387: INFO: PROGRESS: at sentence #2800000, processed 75355112 words, keeping 1441238 word types
2019-08-05 00:31:07,508: INFO: PROGRESS: at sentence #2810000, processed 75570283 words, keeping 1444334 word types
2019-08-05 00:31:07,615: INFO: PROGRESS: at sentence #2820000, processed 75754439 words, keeping 1446383 word types
2019-08-05 00:31:07,738: INFO: PROGRESS: at sentence #2830000, processed 75975049 words, keeping 1449137 word types
2019-08-05 00:31:07,867: INFO: PROGRESS: at sentence #2840000, processed 76202405 words, keeping 1452150 word types
2019-08-05 00:31:07,983: INFO: PROGRESS: at sentence #2850000, processed 76410731 words, keeping 1454975 word types
2019-08-05 00:31:08,118: INFO: PROGRESS: at sentence #2860000, processed 76657602 words, keeping 1458105 word types
2019-08-05 00:31:08,242: INFO: PROGRESS: at sentence #2870000, processed 76883057 words, keeping 1460789 word types
2019-08-05 00:31:08,381: INFO: PROGRESS: at sentence #2880000, processed

...

2019-08-05 00:42:01,653: INFO: EPOCH 2 - PROGRESS: at 22.82% examples, 287963 words/s, in_qsize 8, out_qsize 0
2019-08-05 00:42:02,669: INFO: EPOCH 2 - PROGRESS: at 23.03% examples, 288044 words/s, in_qsize 7, out_qsize 0
2019-08-05 00:42:03,707: INFO: EPOCH 2 - PROGRESS: at 23.22% examples, 287958 words/s, in_qsize 8, out_qsize 0
2019-08-05 00:42:04,731: INFO: EPOCH 2 - PROGRESS: at 23.42% examples, 288089 words/s, in_qsize 3, out_qsize 0
2019-08-05 00:42:05,734: INFO: EPOCH 2 - PROGRESS: at 23.60% examples, 288068 words/s, in_qsize 3, out_qsize 0
2019-08-05 00:42:06,755: INFO: EPOCH 2 - PROGRESS: at 23.81% examples, 288198 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:42:07,820: INFO: EPOCH 2 - PROGRESS: at 24.01% examples, 288060 words/s, in_qsize 7, out_qsize 0
2019-08-05 00:42:08,838: INFO: EPOCH 2 - PROGRESS: at 24.21% examples, 288192 words/s, in_qsize 6, out_qsize 0
2019-08-05 00:42:09,870: INFO: EPOCH 2 - PROGRESS: at 24.40% examples, 288120 words/s, in_qsize 7, out_qsize 0
2

2019-08-05 00:43:17,254: INFO: EPOCH 2 - PROGRESS: at 38.30% examples, 288654 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:43:18,293: INFO: EPOCH 2 - PROGRESS: at 38.54% examples, 288670 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:43:19,301: INFO: EPOCH 2 - PROGRESS: at 38.75% examples, 288642 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:43:20,315: INFO: EPOCH 2 - PROGRESS: at 38.98% examples, 288727 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:43:21,337: INFO: EPOCH 2 - PROGRESS: at 39.17% examples, 288681 words/s, in_qsize 1, out_qsize 0
2019-08-05 00:43:22,356: INFO: EPOCH 2 - PROGRESS: at 39.36% examples, 288721 words/s, in_qsize 2, out_qsize 0
2019-08-05 00:43:23,404: INFO: EPOCH 2 - PROGRESS: at 39.55% examples, 288640 words/s, in_qsize 8, out_qsize 0
2019-08-05 00:43:24,433: INFO: EPOCH 2 - PROGRESS: at 39.75% examples, 288710 words/s, in_qsize 4, out_qsize 1
2019-08-05 00:43:25,457: INFO: EPOCH 2 - PROGRESS: at 39.96% examples, 288663 words/s, in_qsize 7, out_qsize 0
2

2019-08-05 00:44:33,427: INFO: EPOCH 2 - PROGRESS: at 56.84% examples, 284041 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:44:34,462: INFO: EPOCH 2 - PROGRESS: at 57.18% examples, 283883 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:44:35,482: INFO: EPOCH 2 - PROGRESS: at 57.66% examples, 283761 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:44:36,495: INFO: EPOCH 2 - PROGRESS: at 58.19% examples, 283740 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:44:37,540: INFO: EPOCH 2 - PROGRESS: at 58.71% examples, 283620 words/s, in_qsize 0, out_qsize 1
2019-08-05 00:44:38,542: INFO: EPOCH 2 - PROGRESS: at 59.14% examples, 283420 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:44:39,738: INFO: EPOCH 2 - PROGRESS: at 59.50% examples, 283078 words/s, in_qsize 1, out_qsize 0
2019-08-05 00:44:40,765: INFO: EPOCH 2 - PROGRESS: at 59.83% examples, 283142 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:44:41,772: INFO: EPOCH 2 - PROGRESS: at 60.13% examples, 283068 words/s, in_qsize 0, out_qsize 0
2

2019-08-05 00:45:49,691: INFO: EPOCH 2 - PROGRESS: at 75.48% examples, 279917 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:45:50,696: INFO: EPOCH 2 - PROGRESS: at 75.74% examples, 279975 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:45:51,761: INFO: EPOCH 2 - PROGRESS: at 75.94% examples, 279912 words/s, in_qsize 1, out_qsize 0
2019-08-05 00:45:52,811: INFO: EPOCH 2 - PROGRESS: at 76.20% examples, 279960 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:45:53,833: INFO: EPOCH 2 - PROGRESS: at 76.42% examples, 280005 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:45:54,866: INFO: EPOCH 2 - PROGRESS: at 76.61% examples, 279968 words/s, in_qsize 3, out_qsize 0
2019-08-05 00:45:55,888: INFO: EPOCH 2 - PROGRESS: at 76.81% examples, 280037 words/s, in_qsize 3, out_qsize 0
2019-08-05 00:45:56,973: INFO: EPOCH 2 - PROGRESS: at 77.02% examples, 280033 words/s, in_qsize 5, out_qsize 1
2019-08-05 00:45:57,996: INFO: EPOCH 2 - PROGRESS: at 77.25% examples, 280051 words/s, in_qsize 3, out_qsize 0
2

2019-08-05 00:47:05,537: INFO: EPOCH 2 - PROGRESS: at 93.22% examples, 284933 words/s, in_qsize 3, out_qsize 0
2019-08-05 00:47:06,558: INFO: EPOCH 2 - PROGRESS: at 93.52% examples, 285166 words/s, in_qsize 0, out_qsize 0
2019-08-05 00:47:07,607: INFO: EPOCH 2 - PROGRESS: at 93.82% examples, 285321 words/s, in_qsize 3, out_qsize 1
2019-08-05 00:47:08,607: INFO: EPOCH 2 - PROGRESS: at 94.09% examples, 285484 words/s, in_qsize 6, out_qsize 0
2019-08-05 00:47:09,622: INFO: EPOCH 2 - PROGRESS: at 94.40% examples, 285721 words/s, in_qsize 6, out_qsize 0
2019-08-05 00:47:10,642: INFO: EPOCH 2 - PROGRESS: at 94.64% examples, 285872 words/s, in_qsize 5, out_qsize 1
2019-08-05 00:47:11,649: INFO: EPOCH 2 - PROGRESS: at 94.88% examples, 286112 words/s, in_qsize 3, out_qsize 0
2019-08-05 00:47:12,655: INFO: EPOCH 2 - PROGRESS: at 95.12% examples, 286354 words/s, in_qsize 3, out_qsize 1
2019-08-05 00:47:13,672: INFO: EPOCH 2 - PROGRESS: at 95.40% examples, 286585 words/s, in_qsize 5, out_qsize 0
2

最终得到两个文件，Gensim中默认格式的词向量文件model，以及原始c版本的词向量文件vector。

save保存的模型，载入之后可以继续在此基础上接着训练（后文会介绍），而format_save保存的模型不能，但有个好处就是如果s设置binary=False则保存后的结果可以直接打开查看

## 词向量结果测试

In [25]:

# encoding:utf8
import gensim
 
if __name__ == '__main__':
 
    model = gensim.models.Word2Vec.load('D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model')
    word1 = u'农业'
    word2 = u'计算机'
    if word1 in model:
        print (u"'%s'的词向量为： " % word1)
        print (model[word1])
    else:
        print (u'单词不在字典中！')
 
    result = model.most_similar(word2)
    print (u"\n与'%s'最相似的词为： " % word2)
    for e in result:
        print ('%s: %f' % (e[0], e[1]))
 
    print (u"\n'%s'与'%s'的相似度为： " % (word1, word2))
    print (model.similarity(word1, word2))
 
    print (u"\n'早餐 晚餐 午餐 中心'中的离群词为： ")
    print (model.doesnt_match(u"早餐 晚餐 午餐 中心".split()))
 
    print (u"\n与'%s'最相似，而与'%s'最不相似的词为： " % (word1, word2))
    temp = (model.most_similar(positive=[u'农业'], negative=[u'计算机'], topn=1))
    print ('%s: %s' % (temp[0][0], temp[0][1]))

2019-08-05 00:54:22,033: INFO: loading Word2Vec object from D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model
2019-08-05 00:54:23,295: INFO: loading wv recursively from D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model.wv.* with mmap=None
2019-08-05 00:54:23,297: INFO: loading vectors from D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model.wv.vectors.npy with mmap=None
2019-08-05 00:54:24,329: INFO: setting ignored attribute vectors_norm to None
2019-08-05 00:54:24,331: INFO: loading vocabulary recursively from D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model.vocabulary.* with mmap=None
2019-08-05 00:54:24,332: INFO: loading trainables recursively from D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model.trainables.* with mmap=None
2019-08-05 00:54:24,334: INFO: loading syn1neg from D:\code_project\data\wiki_corpus\model_generate\wiki.zh.text.model.trainables.syn1neg.npy with mmap=None
2019-08-05 00:54:25

'农业'的词向量为： 


D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()


[ 4.36175793e-01 -5.61470091e-01 -1.71050608e+00  8.14571977e-01
  4.90316264e-02 -1.85946238e+00 -1.60579109e+00 -3.94210875e-01
 -2.32586920e-01 -5.46416402e-01 -1.45450890e+00 -2.01139688e+00
 -6.05128556e-02 -1.13354576e+00  6.87007532e-02  3.87244463e-01
  7.19080210e-01 -9.00491714e-01  9.44689214e-01 -1.41388595e+00
  5.06428827e-04 -1.77198923e+00 -5.10556340e-01 -7.65648603e-01
 -1.09514749e+00 -1.81155396e+00 -5.44974387e-01 -1.59898782e+00
  2.20470250e-01 -8.85170400e-01  5.46681106e-01 -9.32816148e-01
 -8.46546054e-01  1.27597058e+00  1.54910541e+00  5.46304703e-01
 -8.04875553e-01  2.24322104e+00 -5.73894501e-01  2.38183546e+00
 -2.93526769e+00 -7.70968795e-01 -5.02327621e-01 -1.17211831e+00
 -4.94718671e-01  1.24031043e+00  1.94138801e+00 -4.04175878e-01
  9.07018840e-01  2.09653473e+00 -1.98295832e-01  1.86824262e+00
 -9.85145092e-01  7.95627713e-01  5.52809954e-01  1.87706852e+00
 -6.26223683e-01  6.76890165e-02  2.16248542e-01  4.01196629e-03
 -2.00694174e-01 -8.05163

D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  from ipykernel import kernelapp as app
2019-08-05 00:54:27,637: INFO: precomputing L2-norms of word weight vectors


...

0.21505938466899432

'早餐 晚餐 午餐 中心'中的离群词为： 


D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\gensim\models\keyedvectors.py:730: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


中心

与'农业'最相似，而与'计算机'最不相似的词为： 


D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).


畜牧业: 0.5425076484680176


## 词向量模型可视化
TSNE is pretty useful when it comes to visualizing similarity between objects. It works by taking a group of high-dimensional (100 dimensions via Word2Vec) vocabulary word feature vectors, then compresses them down to 2-dimensional x,y coordinate pairs. The idea is to keep similar words close together on the plane, while maximizing the distance between dissimilar words.

主要思路是：主成分分析降维，然后绘图

In [26]:
from sklearn.manifold import TSNE

def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=1000, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
tsne_plot(model)

D:\xuhui\program64\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
